In [2]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K 
from keras.losses import mse

2024-05-07 13:05:03.778300: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 13:05:03.865460: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 13:05:04.233636: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 13:05:04.238511: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 13:05:05.606265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [ ]:

# Define VAE architecture
# def leaky_relu(x, alpha=0.2):
    # return tf.maximum(alpha * x, x)
def create_vae(input_dim, latent_dim, intermediate_dim,intermediate_dim_1,intermediate_dim_2,intermediate_dim_3):
    input_layer = Input(shape=(input_dim,))
    x = Dense(intermediate_dim, activation='relu')(input_layer)
    x = Dense(intermediate_dim_1, activation='relu')(x)
    x = Dense(intermediate_dim_2, activation='relu')(x)
    x = Dense(intermediate_dim_3, activation='relu')(x)
    # x = Dense(intermediate_dim_4, activation='sigmoid')(x)   
    # x = Dense(intermediate_dim_5, activation='sigmoid')(x)    
    # x = Dense(intermediate_dim_6, activation='sigmoid')(x)

    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)

    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.0, stddev=1.0)
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    z = Lambda(sampling)([z_mean, z_log_var])

    # decoder_h = Dense(intermediate_dim_6, activation='sigmoid')    
    # decoder_h = Dense(intermediate_dim_5, activation='sigmoid')    
    # decoder_h = Dense(intermediate_dim_4, activation='sigmoid')    
    decoder_h = Dense(intermediate_dim_3, activation='relu')
    decoder_h = Dense(intermediate_dim_2, activation='relu')
    decoder_h = Dense(intermediate_dim_1, activation='relu')
    decoder_h = Dense(intermediate_dim, activation='relu')
    decoder_mean = Dense(input_dim, name='decoder_output')

    # Output layer with name 'decoder_output'
    h_decoded = decoder_h(z)
    x_decoded_mean = decoder_mean(h_decoded)

    vae = Model(input_layer, x_decoded_mean)

    reconstruction_loss = mse(K.flatten(input_layer), K.flatten(x_decoded_mean))
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(reconstruction_loss + 0.1*kl_loss)
    # vae_loss = K.mean(reconstruction_loss)    vae.add_loss(vae_loss)
    return vae, z_mean, z_log_var

# Example usage:
input_dim = y_train.shape[1] # Dimension of input and output
latent_dim = 3 # Dimension of latent space
intermediate_dim = 20 # Dimension of intermediate layer
intermediate_dim_1 = 15
intermediate_dim_2 = 10
intermediate_dim_3 = 6
# intermediate_dim_6 = 4
vae, z_mean, z_log_var = create_vae(input_dim, latent_dim, intermediate_dim,intermediate_dim_1,intermediate_dim_2,intermediate_dim_3)
vae.compile(optimizer='adam', metrics = ['R2Score'])
# vae.compile(optimizer='adam', loss='mse', metrics = ['R2Score'])
vae.summary()